In [4]:
import numpy as np
import h5py
import os
from pathlib import Path

from ligotools.readligo import read_hdf5, write_wavfile

def _make_minimal_ligo_hdf5(path: Path, n=1024, fs=4096.0):
    """
    Create a tiny HDF5 with the keys a typical read_hdf5 expects.
    Returns (path, expected_strain, expected_time).
    """
    dt = 1.0 / fs
    t = np.arange(n) * dt
    strain = 0.01 * np.sin(2 * np.pi * 30 * t).astype(np.float64)  # small sine

    with h5py.File(path, "w") as f:
        g = f.create_group("strain")
        d = g.create_dataset("Strain", data=strain)
        # Common attribute for sample spacing used in many tutorials
        d.attrs["Xspacing"] = dt
        # Minimal metadata fields some solutions read
        f.attrs["Detector"] = np.string_("H1")
    return path, strain, t

def test_read_hdf5_returns_arrays_and_right_length(tmp_path):
    h5file = tmp_path / "mini.hdf5"
    _, exp_strain, exp_time = _make_minimal_ligo_hdf5(h5file, n=2048, fs=4096.0)

    out = read_hdf5(str(h5file))

    if isinstance(out, tuple) or isinstance(out, list):
        arrays = [x for x in out if isinstance(x, np.ndarray) and x.ndim == 1]
        assert arrays, "read_hdf5 should return at least one 1D array"
        # expect at least strain array length
        assert any(a.shape == exp_strain.shape for a in arrays)
    else:
        assert isinstance(out, np.ndarray) and out.ndim == 1
        assert out.shape == exp_strain.shape

def test_write_wavfile_roundtrip(tmp_path):
    fs = 8000
    t = np.linspace(0, 0.25, int(0.25*fs), endpoint=False)
    x = (0.5*np.sin(2*np.pi*440*t)).astype(np.float32)  # 440 Hz tone
    wav = tmp_path / "tone.wav"

    write_wavfile(str(wav), fs, x)
    assert wav.exists(), "WAV was not written"

    from scipy.io.wavfile import read as wavread
    fs2, x2 = wavread(str(wav))
    assert fs2 == fs
    assert len(x2) == len(x)


ModuleNotFoundError: No module named 'ligotools'